In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import xlsxwriter

# Load Excel and calculate yearly revenue
df = pd.read_excel("SaaS_Model_Final_With_Chart.xlsx")
df["Year"] = pd.to_datetime(df["Month"]).dt.year
yearly = df.groupby("Year")["Total Revenue"].sum().reset_index()

# DCF assumptions
discount_rate = 0.10
terminal_growth = 0.03

yearly["Discount Factor"] = 1 / (1 + discount_rate) ** (yearly["Year"] - yearly["Year"].min())
yearly["Forecasted Cash Flow"] = yearly["Total Revenue"] * yearly["Discount Factor"]

last_cf = yearly["Total Revenue"].iloc[-1]
terminal_value = last_cf * (1 + terminal_growth) / (discount_rate - terminal_growth)
terminal_discount = 1 / (1 + discount_rate) ** (yearly["Year"].iloc[-1] - yearly["Year"].min())
discounted_terminal_value = terminal_value * terminal_discount

terminal_row = pd.DataFrame({
    "Year": [yearly["Year"].iloc[-1] + 1],
    "Total Revenue": [terminal_value],
    "Discount Factor": [terminal_discount],
    "Forecasted Cash Flow": [discounted_terminal_value]
})
final = pd.concat([yearly, terminal_row], ignore_index=True)
final["Year"] = final["Year"].replace({final["Year"].max(): "Terminal Value"})

npv = final["Forecasted Cash Flow"].sum()
summary_row = pd.DataFrame({
    "Year": ["NPV"],
    "Total Revenue": [""],
    "Discount Factor": [""],
    "Forecasted Cash Flow": [npv]
})
final = pd.concat([final, summary_row], ignore_index=True)

# Plot chart with cleaner labels
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(yearly["Year"], yearly["Total Revenue"] / 1e6, marker='o', label="Total Revenue")
ax.set_title("Forecasted Total Revenue by Year")
ax.set_xlabel("Year")
ax.set_ylabel("Revenue (Millions USD)")
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:.1f}M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
ax.grid(True)
ax.legend()
plt.tight_layout()

# Save plot
chart_path = "revenue_chart.png"
plt.savefig(chart_path)
plt.close()

# Write to Excel
output_file = "saas_dcf_business_ready.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    final.to_excel(writer, sheet_name="DCF", index=False, startrow=2)

    workbook = writer.book
    worksheet = writer.sheets["DCF"]
    worksheet.write("A1", "Forecasted cash flows from SaaS business (in USD), discounted using DCF method")

    # Apply formatting
    currency_format = workbook.add_format({'num_format': '$#,##0', 'align': 'right'})
    percent_format = workbook.add_format({'num_format': '0.0%', 'align': 'right'})
    bold_format = workbook.add_format({'bold': True})

    # Format relevant columns
    worksheet.set_column("B:B", 15, currency_format)
    worksheet.set_column("C:C", 18, percent_format)
    worksheet.set_column("D:D", 22, currency_format)
    worksheet.set_column("A:A", 15)

    # Bold headers
    worksheet.set_row(2, None, bold_format)

    # Insert chart image
    worksheet.insert_image("F3", chart_path)

print(f"\n Excel file generated: {output_file}")



 Excel file generated: saas_dcf_business_ready.xlsx
